<a href="https://colab.research.google.com/github/jing-zhou/w2v/blob/master/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Environment setup and library import**

In [6]:
import random                           # Generates random numbers
import os                               # Create directories, list files
from zipfile import ZipFile
from shutil import copyfile             # Copy files from Source to Destination
import cv2                              # To flip images in data augmentation
import matplotlib.pyplot as plt         # To save the images
import matplotlib.image as mpimg

**2. Link Goggle Drive with Colab**

Run the code and follow the link to get an authentification key, copy it and paste in the box that will appear in jupyter notebook. After fist key the script will ask for a second authentification key, follow the process as above.

Source : https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d 

In [7]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Connection to Google drive successful


In [8]:
# function for handling zip file
def handleZip(corp, zfile, temp):
  # Create a ZipFile Object and load sample.zip in it
  with ZipFile(zfile, 'r') as zipObj:
    # Extract all the contents of zip file in different directory
    zipObj.extractall(temp)
    # Get a list of all archived file names from the zip
    listOfFileNames = zipObj.namelist()
    # Iterate over the file names
    for fileName in listOfFileNames:
      if fileName.endswith('.zip'):
        handleZip(corp, temp + '/' + fileName, temp)
        continue

      tfile = open(temp + '/' + fileName, 'r')
      corp.write(tfile.read())
      tfile.close()



**3. Create root dataset directory and corpus file**

In [9]:
dataset_dir = 'dataset'
if not os.path.exists(dataset_dir):
  os.mkdir(dataset_dir)
  
corpus_file = '/corpus.txt'
# remove previous file instance
if os.path.exists(dataset_dir + corpus_file):
  os.remove(dataset_dir + corpus_file)

corpus = open(dataset_dir + corpus_file, 'x')
# corpus.write("Now the file has more content!")
# corpus.close()
# !cat /content/dataset/corpus.txt

temp_dir = 'temp'
if not os.path.exists(temp_dir):
  os.mkdir(temp_dir)

gdrive = "GDrive/My Drive/corpus/segmented/"

file_list = os.listdir (gdrive)
# print(file_list)

for fname in file_list:
  if fname.endswith('.zip'):
    handleZip(corpus, gdrive + fname, temp_dir)
    continue

  tmp = open(gdrive + fname, 'r')
  corpus.write(tmp.read())
  tmp.close()

corpus.close()
# !cat /content/dataset/corpus.txt


Streaming output truncated to the last 5000 lines.
长鞭 红景天 
长鞭 红景天 （ 学名 ： ） 为 景天科 红景天 属 的 植物 。 分布 于 不丹 、 克什米尔 地区 、 锡金 、 尼泊尔 以及 中国 大陆 的 云南 、 四川 、 西藏 等地 ， 生长 于海拔 2,500 米 至 5,400 米 的 地区 ， 多生 于 山坡石 上 ， 目前 尚 未 由 人工 引种 栽培 。 
宽 叶红 景天 （ 植物 分类学 报 增刊 ） 竖 枝 景天 、 大理 景天 （ 拉汉 种子 植物 名称 ） 
长圆 红景天 
长圆 红景天 （ 学名 ： ） 为 景天科 红景天 属 的 植物 ， 为 中国 的 特有 植物 。 分布 于 中国 大陆 的 云南 、 四川 等地 ， 生长 于海拔 2,900 米 至 4,000 米 的 地区 ， 常 生长 在 山坡 上 ， 目前 尚 未 由 人工 引种 栽培 。 
川滇 景天 、 少 花 景天 （ 拉汉 种子 植物 名称 ） 
长鳞 红景天 
长鳞 红景天 （ 学名 ： ） 是 景天科 红景天 属 的 植物 。 分布 于 蒙古 、 俄罗斯 以及 中国 大陆 的 新疆 等地 ， 生长 于海拔 2,870 米 至 4,180 米 的 地区 ， 一般 生长 在 山坡 草地 上 和 岩石 上 ， 目前 尚 未 由 人工 引种 栽培 。 
孱弱 马先蒿 
孱弱马先蒿 （ 学名 ： ） 为 列当 科马先 蒿属 的 植物 ， 是 中国 的 特有 植物 。 分布 于 中国 大陆 的 云南 等地 ， 生长 于海拔 3,000 米 的 地区 ， 见于 开旷 及 多 沙 的 地方 ， 目前 尚 未 由 人工 引种 栽培 。 
俅江 飞蓬 
俅江 飞蓬 （ 学名 ： ） 为 菊科 飞蓬 属 的 植物 ， 为 中国 的 特有 植物 。 分布 在 中国 大陆 的 云南 、 西藏 等地 ， 生长 于海拔 3,000 米 至 3,200 米 的 地区 ， 一般 生长 在开旷 山坡 或 多 岩石 处 ， 目前 尚 未 由 人工 引种 栽培 。 
硕大 马 先蒿 
硕大 马 先蒿 （ 学名 ： ） 是 列当 科马先 蒿属 的 植物 ， 为 中国 的 特有 植物 。 分布 于 中国 大陆 的 甘肃 、 青海 、 四川 等地 ， 生